In [1]:
import pandas as pd
import numpy as np
import os

# Path to your parquet files
data_dir = "../data/raw"

# List all parquet files
files = [f for f in os.listdir(data_dir) if f.endswith('.parquet')]
print("Available parquet files:", files)

# Load all three parquet files into a single DataFrame
df_0 = [pd.read_parquet(os.path.join(data_dir, f)) for f in files]
df_0 = pd.concat(df_0, ignore_index=True)

df = df_0.dropna(axis=1, how='all')

Available parquet files: ['flight_data_2024_9.parquet', 'flight_data_2025_3.parquet', 'flight_data_2024_8.parquet', 'flight_data_2025_2.parquet', 'flight_data_2025_1.parquet', 'flight_data_2025_4.parquet', 'flight_data_2024_7.parquet', 'flight_data_2024_12.parquet', 'flight_data_2025_5.parquet', 'flight_data_2024_11.parquet', 'flight_data_2025_6.parquet', 'flight_data_2024_10.parquet']


In [ ]:
list_columns = ['Month', 'DayofMonth', 'DayOfWeek', 'OriginAirportID', 'DestAirportID', 
    'Reporting_Airline', 'OriginState', 'DestState', 'CRSDepTime', 'CRSArrTime', 'ArrTime', 
    'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'ArrDelayMinutes']

list_delays = ['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']

list_states = ['AK','AL','AR','AZ','CA','CO','CT','DC','DE','FL','GA','HI','IA','ID','IL','IN',
    'KS','KY','LA','MA','MD','ME','MI','MN','MO','MS','MT','NC','ND','NE','NH','NJ','NM','NV','NY',
    'OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VA','VT','WA','WI','WV','WY']

df = df[list_columns].drop_duplicates()

df[list_delays] = df[list_delays].fillna(0)

df_state_filter = df[df['OriginState'].isin(list_states) & df['DestState'].isin(list_states)].copy()

df_state_filter.loc[:, 'delay_bin'] = np.where(df_state_filter['ArrDelayMinutes'] == 0, 'N', 'Y')

df_state_filter

In [ ]:

# Count how many times each airport appears as origin
origin_counts = df_state_filter['OriginAirportID'].value_counts()

# Count how many times each airport appears as destination
dest_counts = df_state_filter['DestAirportID'].value_counts()

# Combine counts by adding them together, fill missing with 0 before adding
combined_counts = origin_counts.add(dest_counts, fill_value=0)

# Get top 200 airports by combined count
top_200_airports = combined_counts.nlargest(200).index

# Filter rows where both origin and destination are in the top 200 airports
airport_filter_df = df_state_filter[
    (df_state_filter['OriginAirportID'].isin(top_200_airports)) &
    (df_state_filter['DestAirportID'].isin(top_200_airports))
].copy()

airport_filter_df

In [10]:
df_0[df_0[['Year', 'Month', 'DayofMonth']].isna().any(axis=1)]

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,...,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109


In [78]:

import pandas as pd
import numpy as np
import holidays
import os

#############################################################################################################
### FUNCTIONS
#############################################################################################################

### ---------------------------------------------------------------------------------------------------------
### Load parquet files (should have 12 months worth of data)
### ---------------------------------------------------------------------------------------------------------

def load_parquet_files(directory):
    files = [f for f in os.listdir(directory) if f.endswith('.parquet')]
    print("Available parquet files (" + str(len(files)) + "): ", files)
    df = pd.concat([pd.read_parquet(os.path.join(directory, f)) for f in files], ignore_index=True)
    df.columns = df.columns.str.strip().str.replace(' ', '_').str.lower()
    return df

### ---------------------------------------------------------------------------------------------------------
### General data cleaning (remove duplicates, fill NAs, etc)
### ---------------------------------------------------------------------------------------------------------

def clean_and_filter_columns(df, columns, delay_cols):
    df = df.dropna(axis=1, how='all')
    df = df[[col for col in columns if col in df.columns]].drop_duplicates()
    df[delay_cols] = df[delay_cols].fillna(0)
    df[delay_cols] = df[delay_cols].astype(int)
    return df

### ---------------------------------------------------------------------------------------------------------
### Convert departure / arrival times from HHMM (ex. 1420 for 14:20 PM) to minutes after midnight (ex. 860)
### ---------------------------------------------------------------------------------------------------------

def extract_minutes_after_midnight(df, colname, new_colname):
    df[colname] = pd.to_numeric(df[colname], errors='coerce')  # convert to numeric, NaNs if invalid
    hours = df[colname] // 100
    minutes = df[colname] % 100
    df[new_colname] = hours * 60 + minutes
    return df

### ---------------------------------------------------------------------------------------------------------
### Filter to only 50 US states & DC (excludes Canadian and other US territories)
### ---------------------------------------------------------------------------------------------------------

def filter_valid_states(df, valid_states):
    return df[df['originstate'].isin(valid_states) & df['deststate'].isin(valid_states)].copy()

### ---------------------------------------------------------------------------------------------------------
### Filter to top 200 airports based on combined arrival and departures
### ---------------------------------------------------------------------------------------------------------

def get_top_airports(df, n=200):
    origin = df['origin'].value_counts()
    dest = df['dest'].value_counts()
    combined = origin.add(dest, fill_value=0)
    return combined.nlargest(n).index

def filter_by_top_airports(df, top_airports):
    return df[
        df['origin'].isin(top_airports) & df['dest'].isin(top_airports)
    ].copy()

### ---------------------------------------------------------------------------------------------------------
### Create features based on proximity to holidays
### ---------------------------------------------------------------------------------------------------------

def add_holiday_features(df, max_window=14, sentinel=99):
    # Step 1: Add a datetime column
    date_cols = df[['year', 'month', 'dayofmonth']].copy()
    date_cols.rename(columns={'dayofmonth': 'day'}, inplace=True)
    df['flight_date'] = pd.to_datetime(date_cols)

    # Step 2: Define relevant US holidays
    years = df['year'].unique()
    us_holidays = holidays.US(years=years)

    major_holidays = {
        "New Year's Day",
        "Memorial Day",
        "Independence Day",
        "Labor Day",
        "Thanksgiving",
        "Christmas Day"
    }

    filtered_holidays = {date: name for date, name in us_holidays.items() if name in major_holidays}
    holiday_dates = sorted(filtered_holidays.keys())

    # Step 3: Calculate proximity to nearest holiday
    def get_days_from_nearest_holiday(date):
        closest_delta = None
        for holiday in holiday_dates:
            delta = (date.date() - holiday).days
            if abs(delta) <= max_window:
                if (closest_delta is None) or (abs(delta) < abs(closest_delta)):
                    closest_delta = delta
        return closest_delta

    df['days_from_holiday_temp'] = df['flight_date'].apply(get_days_from_nearest_holiday)
    df['if_near_holiday'] = df['days_from_holiday_temp'].notna().astype(int)
    df['days_from_holiday'] = df['days_from_holiday_temp'].fillna(sentinel).astype(int)

    return df


#############################################################################################################
### CALL MAIN
#############################################################################################################

if __name__ == "__main__":
    cols = ['year', 'month', 'dayofmonth', 'dayofweek', 'origin', 'dest', 'reporting_airline', 
        'originstate', 'deststate', 'crsdeptime', 'crsarrtime','carrierdelay', 'weatherdelay', 
        'nasdelay', 'securitydelay', 'lateaircraftdelay', 'arrdelayminutes', 'cancelled', 'diverted']
    delay_cols = ['carrierdelay', 'weatherdelay', 'nasdelay', 'securitydelay', 
        'lateaircraftdelay', 'arrdelayminutes']
    state_list = ['AK','AL','AR','AZ','CA','CO','CT','DC','DE','FL','GA','HI','IA','ID','IL','IN',
        'KS','KY','LA','MA','MD','ME','MI','MN','MO','MS','MT','NC','ND','NE','NH','NJ','NM','NV',
        'NY','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VA','VT','WA','WI','WV','WY']

    df_raw = load_parquet_files("../data/raw")
    df_clean = clean_and_filter_columns(df_raw, cols, delay_cols)
    df_filtered = filter_valid_states(df_clean, state_list)
    df_filtered = df_filtered.drop(columns=['originstate', 'deststate'])
    df_filtered = extract_minutes_after_midnight(df_filtered, 'crsdeptime', 'deptime_mins')
    df_filtered = extract_minutes_after_midnight(df_filtered, 'crsarrtime', 'arrtime_mins')
    df_filtered = df_filtered.drop(columns=['crsdeptime', 'crsarrtime'])
    df_filtered = add_holiday_features(df_filtered)
    df_filtered = df_filtered.drop(columns=['year', 'flight_date', 'days_from_holiday_temp'])
    df_filtered['if_delay'] = np.where(df_filtered['arrdelayminutes'] <= 15, '0', '1').astype(int) # delays defined as more than 15 minutes
    df_filtered['if_cancelled'] = np.where(df_filtered['cancelled'] == 0, '0', '1').astype(int)
    df_filtered['if_diverted'] = np.where(df_filtered['diverted'] == 0, '0', '1').astype(int)
    df_filtered = df_filtered.drop(columns=['cancelled', 'diverted'])
    top_airports = get_top_airports(df_filtered)
    df_final = filter_by_top_airports(df_filtered, top_airports)


#############################################################################################################
### SUMMARY STATISTICS DATASET
#############################################################################################################

summary_cols = ['origin', 'dest', 'reporting_airline', 'month', 'dayofweek', 'if_near_holiday']

df_summary = df_final.copy().groupby(summary_cols).agg(
    total_flights = ('if_delay', 'count'),
    delayed_flights = ('if_delay', 'sum'),
    cancelled_flights = ('if_cancelled', 'sum'),
    diverted_flights = ('if_diverted', 'sum'),
    total_delay_minutes = ('arrdelayminutes', 'sum')
).reset_index()

df_summary.to_parquet('../data/processed/summary_dataset')

#############################################################################################################
### MACHINE LEARNING DATASET
#############################################################################################################

df_ml = df_final.copy()


Available parquet files (12):  ['flight_data_2024_9.parquet', 'flight_data_2025_3.parquet', 'flight_data_2024_8.parquet', 'flight_data_2025_2.parquet', 'flight_data_2025_1.parquet', 'flight_data_2025_4.parquet', 'flight_data_2024_7.parquet', 'flight_data_2024_12.parquet', 'flight_data_2025_5.parquet', 'flight_data_2024_11.parquet', 'flight_data_2025_6.parquet', 'flight_data_2024_10.parquet']


In [93]:

summary_sandbox_cols = ['origin']

df_summary_sandbox = df_final.copy().groupby(summary_sandbox_cols).agg(
    total_flights = ('if_delay', 'count'),
    delayed_flights = ('if_delay', 'sum'),
    cancelled_flights = ('if_cancelled', 'sum'),
    diverted_flights = ('if_diverted', 'sum'),
    total_delay_minutes = ('arrdelayminutes', 'sum'),
    delay_minutes_90th = ('arrdelayminutes', lambda x: int(x.quantile(0.90))),
    delay_minutes_95th = ('arrdelayminutes', lambda x: int(x.quantile(0.95))),
    delay_minutes_99th = ('arrdelayminutes', lambda x: int(x.quantile(0.99))),
).reset_index()

df_summary_sandbox['avg_delay'] = df_summary_sandbox['total_delay_minutes'] / df_summary_sandbox['delayed_flights']
df_summary_sandbox['delay_percent'] = (100 * df_summary_sandbox['delayed_flights'] / df_summary_sandbox['total_flights']).round(1).astype(str) + '%'
df_summary_sandbox['cancelled_percent'] = (100 * df_summary_sandbox['cancelled_flights'] / df_summary_sandbox['total_flights']).round(1).astype(str) + '%'
df_summary_sandbox['diverted_percent'] = (100 * df_summary_sandbox['diverted_flights'] / df_summary_sandbox['total_flights']).round(1).astype(str) + '%'

df_summary_sandbox.sort_values(by='delay_minutes_90th').tail(25)


,origin,total_flights,delayed_flights,cancelled_flights,diverted_flights,total_delay_minutes,delay_minutes_90th,delay_minutes_95th,delay_minutes_99th,avg_delay,delay_percent,cancelled_percent,diverted_percent
181,SHV,5173,1058,128,8,101707,56,106,257,96.131380,20.5%,2.5%,0.2%
126,MIA,106461,25533,1539,325,2048857,56,100,242,80.243489,24.0%,1.4%,0.3%
149,PHL,98682,22818,2046,241,1949909,57,104,254,85.454860,23.1%,2.1%,0.2%
179,SGF,9696,2305,133,19,219769,57,120,315,95.344469,23.8%,1.4%,0.2%
195,TVC,4476,1115,45,9,93187,58,101,244,83.575785,24.9%,1.0%,0.2%
61,EWR,120136,27370,2870,277,2307528,58,107,247,84.308659,22.8%,2.4%,0.2%
51,DFW,295609,74824,6553,780,5870459,58,101,234,78.456899,25.3%,2.2%,0.3%
151,PIA,4591,1053,62,6,108071,59,113,336,102.631529,22.9%,1.4%,0.1%
146,PBI,29974,6922,471,109,615294,59,111,265,88.889627,23.1%,1.6%,0.4%
113,LNK,2560,546,28,11,55333,60,126,314,101.342491,21.3%,1.1%,0.4%
